In [2]:
import requests
import numpy as np
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00


In [3]:
!pip install faiss-gpu
import faiss

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 3.1 MB/s eta 0:00:00


In [4]:
!pip install sentence_transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 2.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [5]:
from sentence_transformers import SentenceTransformer, util

In [6]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
dimension = 768  # Dimension of embeddings from Sentence-BERT
index = faiss.IndexFlatL2(dimension)

Retrieve Data From WordPress

In [8]:
api_url = ' https://public-api.wordpress.com/wp/v2/'

def fetch_wp_content(api_url):
  response = requests.get(api_url)
  if response.status_code == 200:
    return response.json()
  else:
    return none

Generate embeddings for content

In [9]:
def generate_embeddings(content_list):
  embeddings = model.encode(content_list, convert_to_tensor=True)
  return embeddings

Add embeddings to Faiss index

In [10]:
def add_embeddings_to_index(embeddings):
  faiss.normalize_L2(embeddings)
  index.add(embeddings)

Search Similar Content

In [11]:
def search_similar_content(query, k=5):
    query_embedding = model.encode([query], convert_to_tensor=True)
    faiss.normalize_L2(query_embedding)
    distances, indices = index.search(query_embedding, k)
    return distances, indices

RAG Processor

In [16]:
def generate_rag_response(query, api_url):
    content = fetch_wp_content(api_url)
    content_list = [item['content'] for item in content]

    embeddings = generate_embeddings(content_list)
    add_embeddings_to_index(embeddings)

    distances, indices = search_similar_content(query)
    similar_content = [content_list[i] for i in indices[0]]

    context = " ".join(similar_content)

    # Chain of Thought processing
    response = chain_of_thought_processing(context, query)
    return response

Chain of Thought Module

In [17]:
def chain_of_thought_processing(context, query):
    openai.api_key = 'your_openai_api_key'
    prompt = f"Context: {context}\nQuery: {query}\n\nAnswer in a logical, step-by-step manner:"
    response = openai.Completion.create(engine="davinci-codex", prompt=prompt, max_tokens=150)
    return response.choices[0].text.strip()

Reranking

In [20]:
def reranker_colbert(query,chunks):
        from transformers import AutoTokenizer,AutoModel, PreTrainedModel,PretrainedConfig
        from typing import Dict
        import torch

        class ColBERTConfig(PretrainedConfig):
            model_type = "ColBERT"
            bert_model: str
            compression_dim: int = 768
            dropout: float = 0.0
            return_vecs: bool = False
            trainable: bool = True

        class ColBERT(PreTrainedModel):
            """
            ColBERT model from: https://arxiv.org/pdf/2004.12832.pdf
            We use a dot-product instead of cosine per term (slightly better)
            """
            config_class = ColBERTConfig
            base_model_prefix = "bert_model"

            def __init__(self,
                        cfg) -> None:
                super().__init__(cfg)

                self.bert_model = AutoModel.from_pretrained(cfg.bert_model)

                for p in self.bert_model.parameters():
                    p.requires_grad = cfg.trainable

                self.compressor = torch.nn.Linear(self.bert_model.config.hidden_size, cfg.compression_dim)

            def forward(self,
                        query: Dict[str, torch.LongTensor],
                        document: Dict[str, torch.LongTensor]):

                query_vecs = self.forward_representation(query)
                document_vecs = self.forward_representation(document)

                score = self.forward_aggregation(query_vecs,document_vecs,query["attention_mask"],document["attention_mask"])
                return score

            def forward_representation(self,
                                    tokens,
                                    sequence_type=None) -> torch.Tensor:

                vecs = self.bert_model(**tokens)[0] # assuming a distilbert model here
                vecs = self.compressor(vecs)

                # if encoding only, zero-out the mask values so we can compress storage
                if sequence_type == "doc_encode" or sequence_type == "query_encode":
                    vecs = vecs * tokens["tokens"]["mask"].unsqueeze(-1)

                return vecs

            def forward_aggregation(self,query_vecs, document_vecs,query_mask,document_mask):

                # create initial term-x-term scores (dot-product)
                score = torch.bmm(query_vecs, document_vecs.transpose(2,1))

                # mask out padding on the doc dimension (mask by -1000, because max should not select those, setting it to 0 might select them)
                exp_mask = document_mask.bool().unsqueeze(1).expand(-1,score.shape[1],-1)
                score[~exp_mask] = - 10000

                # max pooling over document dimension
                score = score.max(-1).values

                # mask out paddding query values
                score[~(query_mask.bool())] = 0

                # sum over query values
                score = score.sum(-1)

                return score

        #
        # init the model & tokenizer (using the distilbert tokenizer)
        #
        tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased") # honestly not sure if that is the best way to go, but it works :)
        model_colbert = ColBERT.from_pretrained("sebastian-hofstaetter/colbert-distilbert-margin_mse-T2-msmarco")
        query_input = tokenizer(query)
        l=[]
        for i,v in enumerate(chunks):

            d = tokenizer(v.page_content,return_tensors="pt",truncation=True)
            l.append(d)


        query_input.input_ids += [103] * 8 # [MASK]
        query_input.attention_mask += [1] * 8
        query_input["input_ids"] = torch.LongTensor(query_input.input_ids).unsqueeze(0)
        query_input["attention_mask"] = torch.LongTensor(query_input.attention_mask).unsqueeze(0)


        l1=[]

        for i in l:

            score_for_p1 = model_colbert.forward(query_input,i).squeeze(0)
            l1.append(float(score_for_p1))

        reranked = sorted(zip(chunks,l1),key=lambda x:-x[1])
        reranked_res, scores = zip(*reranked[:5])  # Unpack the result into reranked_res and scores
        return reranked_res, scores

User Interface


In [21]:
def chat_interface():
    api_url = 'https://public-api.wordpress.com/wp/v2/'
    while True:
        query = input("You: ")
        if query.lower() == 'exit':
            break
        response = generate_rag_response(query, api_url)
        print(f"Chatbot: {response}")

In [ ]:
chat_interface()